# Quantum. Lab 6. Phase estimation (using IQFT)

Author:
- ***Nikita Makarevich (Student ID: 153989)***

In [19]:
import numpy as np
from qiskit import QuantumRegister, ClassicalRegister, QuantumCircuit, transpile
import qiskit.visualization as qvis
from qiskit.quantum_info import Operator
from qiskit_aer import Aer
import matplotlib.pyplot as plt
from qiskit.circuit.library import QFTGate
from dataclasses import dataclass

from qiskit_aer import AerSimulator
from qiskit_ibm_runtime import SamplerV2 as Sampler
from qiskit import transpile
# from qiskit.quantum_info.states.statevector import Statevector

# selection of quantum simulator (or processor)
# backend = Aer.get_backend("qasm_simulator")
backend = Aer.get_backend("aer_simulator")

In [20]:
def plot_circuit(circuit: QuantumCircuit, *, ax: plt.Axes | None = None) -> plt.Axes:
    if ax is None:
        fig = plt.figure(figsize=(8, 4))
        ax = fig.add_subplot(111)
    circuit.draw("mpl", ax=ax)
    return ax

# def plot_bloch_spheres(backend, circuit: QuantumCircuit):
#     circuit_init = circuit.copy()
#     circuit_init.save_statevector()
#     statevector = backend.run(circuit_init).result().get_statevector()
#     # statevector = Statevector(circuit_init)
#     fig: plt.Figure = plot_bloch_multivector(statevector, title="Final State Bloch Spheres")
#     fig.show()
#     display(fig)
#     return fig


def plot_circuit_and_results(
    circuit: QuantumCircuit,
    counts: list[dict],
    *,
    figsize: tuple[float, float] = (15, 8),
) -> plt.Figure:
    fig = plt.figure(figsize=figsize)
    gs = fig.add_gridspec(2, 2)

    axs = [
        fig.add_subplot(gs[0, :]),
        fig.add_subplot(gs[1, 0]),
        fig.add_subplot(gs[1, 1]),
    ]

    plot_circuit(circuit, ax=axs[0])
    qvis.plot_histogram(
        counts,
        ax=axs[1],
    )
    qvis.plot_distribution(
        counts,
        legend=[f"Execution {i + 1}" for i in range(len(counts))],
        ax=axs[2],
    )

    for ax in (axs[1], axs[2]):
        old_texts = ax.texts[:]  # copy list of text objects
        for t in old_texts:
            t.remove()

    for ax in (axs[1], axs[2]):
        for p in ax.patches:  # each bar
            height = p.get_height()
            x = p.get_x() + p.get_width() / 2
            ax.annotate(
                f"{height}" if float(height).is_integer() else f"{height:.2f}",
                (x, height),
                ha="center",
                va="bottom",
                rotation=45,
                fontsize=9,
            )

    axs[0].set_title("Circuit")
    axs[1].set_title("Distribution [counts]")
    axs[2].set_title("Probabilities")
    return fig


def run_experiments(
    backend, circuit: QuantumCircuit, *, shots: int = 2048, runs: int = 3
) -> list[dict]:
    compiled_circuit = transpile(circuit, backend)
    return [
        backend.run(compiled_circuit, shots=shots).result().get_counts()
        for _ in range(runs)
    ]

In [21]:
@dataclass
class Setup:
    n: int
    qx: QuantumRegister
    cx: ClassicalRegister | None
    circuit: QuantumCircuit

    @classmethod
    def create_quantum_only(cls, *, n: int = 3, **circuit_kwargs) -> "Setup":
        qx = QuantumRegister(n, "q")
        circ = QuantumCircuit(qx, **circuit_kwargs)
        return cls(n, qx, None, circ)

    @classmethod
    def create(cls, *, n: int = 3, **circuit_kwargs) -> "Setup":
        qx = QuantumRegister(n, "q")
        cx = ClassicalRegister(n, "c")
        circ = QuantumCircuit(qx, cx, **circuit_kwargs)
        return cls(n, qx, cx, circ)

In [22]:
def create_phase_estimation_circuit(m: int = 10, theta: float = 0.7) -> QuantumCircuit:
    control_register = QuantumRegister(m, name="Control")
    target_register = QuantumRegister(1, name="|ψ>")
    output_register = ClassicalRegister(m, name="Result")
    qc = QuantumCircuit(control_register, target_register, output_register)

    # Prepare the eigenvector |ψ>
    qc.x(target_register)
    qc.barrier()

    # Perform phase estimation
    for index, qubit in enumerate(control_register):
        qc.h(qubit)
        for _ in range(2**index):
            qc.cp(2 * np.pi * theta, qubit, target_register)
    qc.barrier()

    # Do inverse quantum Fourier transform
    qc.compose(
        QFTGate(m).inverse(),
        inplace=True
    )

    return qc

# fig, ax = plt.subplots(1, 1, figsize=(3, 10))
# plot_circuit(qc, ax=ax)



In [23]:
def run_phase_estimation_circuit(qc: QuantumCircuit) -> dict:
    backend = AerSimulator()

    qc_transpiled = transpile(qc, backend)
    sampler = Sampler(mode=backend)
    job = sampler.run([qc_transpiled])

    result = job.result()

    counts = result[0].data.Result.get_counts()

    return counts

In [24]:
qc = create_phase_estimation_circuit(theta=0.7, m=10)
counts = run_phase_estimation_circuit(qc)

In [25]:
counts

{'0000000000': 1024}

In [ ]:
def plot_phase_estimation(ns: list[int], m: int, theta: float):
    fig, axs = plt.subplots(2, 1, figsize=(8, 12))

    ys = []  # TODO
    uncertainties: list[float] = []
    for i in range(len(ys)):
        uncertainty = ((ys[i] - theta) / theta) * 100
        uncertainties.append(uncertainty)

    axs[0].title("Phase estimation $\\theta_e$ (where $\\theta=0.70$)")
    axs[0].xlabel(r" Number of qubits ")
    axs[0].ylabel(r"$\theta_e$")
    axs[0].xlim([0, m + 0.2])
    axs[0].plot(ns, ys, "g^")
    axs[0].hlines(y=theta, xmin=0.0, xmax=m + 0.2, colors="k", linestyles="dashed")

    axs[1].title(
        r"Phase estimation uncertainty $\frac{\theta_e - \theta}{\theta} \cdot 100\%$"
    )
    axs[1].xlabel(r" Number of qubits ")
    axs[1].ylabel(r"$\frac{\theta_e - \theta}{\theta} \cdot 100\%$")
    axs[1].xlim([0, m + 0.2])
    axs[1].plot(ns, uncertainties, "g^")
    axs[1].hlines(y=0.0, xmin=0.0, xmax=m + 0.2, colors="k", linestyles="dashed")


plot_phase_estimation(ns=list(range(1, 11)), m=10, theta=0.70)